In [1]:
# For MacOS:
# https://github.com/mkleehammer/pyodbc/wiki/Connecting-to-SQL-Server-from-Mac-OSX
# `brew install unixodbc freetds`

# For Windows:
# https://github.com/mkleehammer/pyodbc/wiki/Connecting-to-SQL-Server-from-Windows

In [2]:
import platform
from pathlib import Path
import json
from pprint import pprint

import pyodbc

In [3]:
DB_DSN = "ML_Internal_DB"
DB_PORT=1433
DB_USER="sa"
DB_PASSWORD=Path('./.db-password.txt').read_text().strip()
DB_DATABASE="DemoData"

In [4]:
pyodbc.drivers()

['FreeTDS']

In [5]:
if platform.system() == "Windows":
    driver = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=127.0.0.1'
elif platform.system() == "Darwin": # MacOS
    driver = f'DSN={DB_DSN}'
else:
    raise NotImplemented

connection_string = f'{driver};DATABASE={DB_DATABASE};UID={DB_USER};PWD={DB_PASSWORD}'

connection = pyodbc.connect(connection_string)

In [6]:
connection.setencoding(encoding='utf-8')


In [7]:
cursor = connection.cursor()

In [8]:
cursor.execute(
"""
SELECT DB_SCHEMA.TABLE_SCHEMA         "tableSchema",
       Tables.name                    "tableName",
       Columns.name                   "columnName",
       Columns.is_nullable            "isNullable",
       Columns.is_identity            "isIdentity",
       Columns.max_length             "maxLength",
       Type.name                      "dataType",
       Type.precision                 "precision",
       Type.scale                     "scale",
       Type.max_length                "maxLength",
       Default_Constraints.definition "defaultConstraint"

FROM INFORMATION_SCHEMA.TABLES DB_SCHEMA

         JOIN sys.objects "Tables"
              on DB_SCHEMA.TABLE_SCHEMA = schema_name(TABLES.schema_id) AND DB_SCHEMA.TABLE_NAME = TABLES.name

         JOIN sys.columns Columns on Tables.object_id = Columns.object_id

         LEFT JOIN sys.identity_columns id on Tables.object_id = id.object_id

    -- see https://docs.microsoft.com/en-us/sql/relational-databases/system-catalog-views/sys-types-transact-sql
         JOIN sys.types Type
              on Columns.user_type_id = Type.user_type_id

         LEFT JOIN sys.default_constraints Default_Constraints
                   on Columns.default_object_id = Default_Constraints.object_id

-- see https://docs.microsoft.com/en-us/sql/relational-databases/system-catalog-views/sys-columns-transact-sql
WHERE Tables.type in ('U', 'V')

ORDER BY TABLE_SCHEMA, TABLE_NAME


FOR JSON AUTO, root('database')
"""
)

results = cursor.fetchall()
assert len(results) == 1, "Expected single result row"
assert len(results[0]) == 1, "Expected single-value tuple with JSON-formatted string"
print(results[0][0])
result = json.loads(str(results[0][0]))

{"database":[{"tableSchema":"dbo","Tables":[{"tableName":"Products","Columns":[{"columnName":"ID","isNullable":true,"isIdentity":false,"maxLength":4,"Type":[{"dataType":"int","precision":10,"scale":0,"maxLength":4,"Default_Constraints":[{}]}]},{"columnName":"ProductName","isNullable":true,"isIdentity":false,"maxLength":-1,"Type":[{"dataType":"nvarchar","precision":0,"scale":0,"maxLength":8000,"Default_Constraints":[{}]}]}]}]}]}


In [9]:
# Print in Python format
pprint(result, indent=2)

{ 'database': [ { 'Tables': [ { 'Columns': [ { 'Type': [ { 'Default_Constraints': [ { }],
                                                           'dataType': 'int',
                                                           'maxLength': 4,
                                                           'precision': 10,
                                                           'scale': 0}],
                                               'columnName': 'ID',
                                               'isIdentity': False,
                                               'isNullable': True,
                                               'maxLength': 4},
                                             { 'Type': [ { 'Default_Constraints': [ { }],
                                                           'dataType': 'nvarchar',
                                                           'maxLength': 8000,
                                                           'precision': 0,
                

In [10]:
# Print in JSON format
print(json.dumps(result, indent=2))

{
  "database": [
    {
      "tableSchema": "dbo",
      "Tables": [
        {
          "tableName": "Products",
          "Columns": [
            {
              "columnName": "ID",
              "isNullable": true,
              "isIdentity": false,
              "maxLength": 4,
              "Type": [
                {
                  "dataType": "int",
                  "precision": 10,
                  "scale": 0,
                  "maxLength": 4,
                  "Default_Constraints": [
                    {}
                  ]
                }
              ]
            },
            {
              "columnName": "ProductName",
              "isNullable": true,
              "isIdentity": false,
              "maxLength": -1,
              "Type": [
                {
                  "dataType": "nvarchar",
                  "precision": 0,
                  "scale": 0,
                  "maxLength": 8000,
                  "Default_Constraints": [
                 